Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChains
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
df2[:height] = convert(Vector{Float64}, df2[:height]);
df2[:weight] = convert(Vector{Float64}, df2[:weight]);
df2[:weight_s] = (df2[:weight] .- mean(df2[:weight])) / std(df2[:weight]);
df2[:weight_s2] = df2[:weight_s] .^ 2;

Define the Stan language model

In [5]:
weightsmodel = "
data{
    int N;
    real height[N];
    real weight_s2[N];
    real weight_s[N];
}
parameters{
    real a;
    real b1;
    real b2;
    real sigma;
}
model{
    vector[N] mu;
    sigma ~ uniform( 0 , 50 );
    b2 ~ normal( 0 , 10 );
    b1 ~ normal( 0 , 10 );
    a ~ normal( 178 , 100 );
    for ( i in 1:N ) {
        mu[i] = a + b1 * weight_s[i] + b2 * weight_s2[i];
    }
    height ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["a", "b1", "b2", "sigma"],
model=weightsmodel,  output_format=:mcmcchains);


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan will be updated.



Input data for cmdstan

In [7]:
heightsdata = Dict("N" => size(df2, 1), "height" => df2[:height],
"weight_s" => df2[:weight_s], "weight_s2" => df2[:weight_s2]);

Sample using cmdstan

In [8]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is -56.7142, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights.stan' at line 22)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.


Inference for Stan model: weights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.90, 0.90, 1.5, 1.1) seconds, 4.4 seconds total
Sampling took (0.26, 0.25, 0.15, 0.18) seconds, 0.83 seconds total

                    Mean     MCSE  StdDev     5%       50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -7.5e+02  3.3e-02     1.4   -753  -7.5e+02  -748  1.9e+03  2.2e+03  1.0e+00
accept_stat__    9.1e-01 

Describe the draws

In [9]:
describe(chn)

Log evidence      = 0.0
Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
parameters        = a, b1, b2, sigma

Empirical Posterior Estimates
──────────────────────────────────────────────────
parameters
        Mean     SD   Naive SE  MCSE   ESS
    a 154.6041 0.3621   0.0057 0.0063 1000
   b1   5.8421 0.2745   0.0043 0.0034 1000
   b2  -0.0105 0.2267   0.0036 0.0041 1000
sigma   5.1121 0.1986   0.0031 0.0030 1000

Quantiles
──────────────────────────────────────────────────
parameters
        2.5%     25.0%    50.0%    75.0%    97.5% 
    a 153.3840 154.3650 154.6120 154.8480 155.8560
   b1   4.8161   5.6577   5.8465   6.0268   6.7997
   b2  -0.8205  -0.1659  -0.0108   0.1422   0.7986
sigma   4.4474   4.9754   5.1043   5.2430   5.8711



End of `m4.5s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*